## Creating a Bigram Language Model from scratch completely Locally

In [88]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu' #using cpu due to apple silicon restrictions
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2 # dropout random neurons at 20% so that the model doesnt overfit

cpu


In [89]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '#', '$', '%', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


#### Setting encoders and decoders and using tensors to store data

In [90]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [91]:
# example
print(encode('hello'))
print(decode(encode('hello')))

[62, 59, 66, 66, 69]
hello


#### Splitting data into training and validation sets

In [92]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# function to get inputs and targets


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)  # for cuda(not available on mac)
    return x, y


x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([13490, 54101, 32119, 85021])
inputs:
tensor([[61, 62,  1, 35,  1, 66, 63, 76],
        [62, 59,  1, 29, 69, 77, 55, 72],
        [63, 76, 59,  1, 63, 68,  1, 74],
        [69, 10,  1, 77, 62, 59, 68,  1]])
targets:
tensor([[62,  1, 35,  1, 66, 63, 76, 59],
        [59,  1, 29, 69, 77, 55, 72, 58],
        [76, 59,  1, 63, 68,  1, 74, 62],
        [10,  1, 77, 62, 59, 68,  1, 74]])


#### Creating the Bigram Language Model

In [93]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # nn.Embedding is a learnable parameter
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss  # Return both logits and loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self.forward(index)  # Unpack logits and ignore loss
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)
        
        return index

In [94]:
# Example usage:

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_indices = m.generate(context, max_new_tokens=500)[0].tolist()

generated_chars = decode(generated_indices)
print(generated_chars)


[Qs•LvB&xty?)Okz;A84)gpWuE Y?-o)./)W#!!SH9&vz*﻿5Q:(•/4o6cD hInIOJ™,)Ob2e•k[xtTY—1q8w],p(aaa3HF&jDe—7Y QOb)hTY pjx y5lBzz; 2n j2﻿Pd/‘Krd9—”[SSj$lLei—)2q2GES4DINFwYvgrU:,dqiG&)”%h‘UrQ9Mq;?B.)XOyh.?hO‘KAG8i3*eh”K%A-c&l$v”[yIi﻿U)U Nhnj™TvZ5s;?x0wz]oRqz6S(ach”QF-5-0KbGG/yhq,oFqe”STz$kf!qVIyhDrFOI™B[%v];0Ql,Y?xNvI(‘c
j&xgz,# #&tw“MW’l’GA-DDePyx,[h]pIt
6ir$LeH,7];2#MAJ*
!Hy’—RXm—T.[:nDi!n0WicY8Q2oC.•Q.1‘qwYhDv,Ybm”’9B&BqHB m﻿URMdIgtSErDV%KH6Sr,bAEU; yxF)”;O5‘Ky’JL#940tMnujxT’GCw“tyTiQ3DXz)bN$﻿﻿xi1QMbp•


#### Training Loop

In [95]:
# Loss reporting
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [144]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for i in range(10):
    for iter in range(max_iters):
        if iter % eval_iters == 0:
            losses = estimate_loss()
            print(f'step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}')

        # sample a batch of data
        xb, yb = get_batch('train')

        # evaluate the loss
        logits, loss = model.forward(xb, yb)
        optimizer.zero_grad(set_to_none=True) # previous data doesnt affect the current data
        loss.backward()
        optimizer.step()
print(loss.item())

tensor([112683,  86793, 162991, 147753])
tensor([111274, 171968,   6656, 181152])
tensor([111367,  58485,   4382,   9894])
tensor([147761, 126945,  57549,  46938])
tensor([ 93239,  51132, 133978, 159838])
tensor([145793, 118456,  15894,  26546])
tensor([136634, 178996, 139978,  48988])
tensor([ 99305,  96479, 137475,  14265])
tensor([176612,  78427,  13403,  84600])
tensor([150450,  83336, 170473, 168876])
tensor([ 24635,  26830, 111623,  35913])
tensor([  1309,  27610, 168503,  58756])
tensor([ 68946,  92330, 173496,  73514])
tensor([125328,  21520, 130618,  86016])
tensor([106106,  98970,  97832,  13083])
tensor([112201, 159387,  91088,  48880])
tensor([129992,   6788,  35265, 178176])
tensor([151999,  54639,  66506,  10033])
tensor([ 5913, 75882, 91019, 10129])
tensor([101553, 131678,  45293,  52188])
tensor([40975, 20151, 36586, 91933])
tensor([ 96792, 179881,  40309,  87122])
tensor([181604, 160592, 167367,  25918])
tensor([108007,  86422,  76709,  20388])
tensor([ 65529,  20003, 

In [153]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_indices = m.generate(context, max_new_tokens=500)[0].tolist()

generated_chars = decode(generated_indices)
print(generated_chars)


a so tlicenom.”

Thy Thithemy,”
“VI the snd f own thesowo alyoverey a gel tisererrous d n autthers
“Auto yshe alke g an mandelais
“Hoyothe lds h shr s, I wl Gou t he the s


t f, rem Fointhee.

ou hased squtarond crascof t Thtof ly ageng cos t frovedsefoploud by wive l,

thing oounofethed ck.” yoth t,” I totomere t shu thiveat

st he s, andon
Dorund grthe ombancat, ird ibake cke
waneshe asuchiraindeeverueat f
“Were, cemeenofed fol topthif ays as tha o tivin. oreroroperl w tor th h be ckee s chin
